<a href="https://colab.research.google.com/github/PavanDaniele/drone-person-detection/blob/main/model_Training_and_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up: mount drive + import libraries

**Important Information:** We need to activate the GPU on Colab (_Runtime --> Change runtime type_). \
Every time you start a new session (or reopen the notebook after a few hours) check that the GPU is still active. If we are not using the GPU it can take up to tens of hours to train the models. \
_GPU T4 is the best choice._

In [2]:
# Run this Every time you start a new session
from google.colab import drive
drive.mount('/content/drive') # to mount google drive (to see/access it)

Mounted at /content/drive


In [3]:
!pip install ultralytics # Installation of Ultralytics for YOLO models

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [4]:
from ultralytics import YOLO # Import of Ultralytics for YOLO models

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# General Explanation

### Backbone

In Computer Vision, a _Backbone_ is the part of a convolutional neural network responsible for extracting the main features from an image. \
It serves as the shared base upon which subsequent modules are built (such as heads for classification, object detection, segmentation, etc.).

\
Each backbone has been pre-trained on specific datasets (e.g., ImageNet) using particular preprocessing steps, input dimensions, normalization, and augmentation techniques, which should ideally be replicated during fine-tuning to maintain compatibility and achieve optimal performance.

### Data Loader

To train a deep learning model, it is essential to properly handle data loading and preparation. This is the task of the _Data Loader_, a component responsible for:
- Loading images and their corresponding annotations (e.g., .txt or .json) from the dataset.
- Applying preprocessing operations, such as resizing, normalization, data augmentation, etc.
- Organizing data into batches to feed the model during training.

\
Considering the limited resources of my development environment, at first I decided to perform the image and annotation resizing in a separate phase (prior to training), in order to:
- Reduce the workload of the data loader during fine-tuning;
- Increase data loading and training speed;
- Ensure consistency between images and annotations.

But due to the different type of scaling technique, I want to try to fine-tuning the model without any pre-scaling.

The other transformations, instead, are handled by the model-specific data loader, since each model uses different preprocessing and normalization techniques. \
Moreover, some models require specific transformations to achieve optimal performance, and the libraries that provide the models (e.g., Ultralytics for YOLO, torchvision for EfficientDet/SSD) already implement loaders that are properly configured and optimized.

### Normalizzazione

Image normalization consists in scaling pixel values from the range [0, 255] to a more suitable interval (e.g., [0, 1] or [-1, 1]), often based on the mean and standard deviation of the pre-training dataset, with the goal of:
- Avoiding overly large values in the tensors;
- Making the model more stable during training;
- Speeding up convergence.

\
Normalization helps maintain a consistent pixel range and distribution, which is essential for pre-trained models.

### Data Augmentations

Data augmentation consists of random transformations (e.g., rotations, flips, crops, brightness changes, etc.) applied during training. Their purpose is to:
- Simulate new visual conditions;
- Increase dataset variety;
- Reduce overfitting by improving the model’s ability to generalize.

\
In practice, the semantic content of the image doesn't change (e.g., a person remains a person), but its visual appearance is altered to help the model "learn better."


My goal is to evaluate the real-world performance of each model in its ideal scenario, in order to select the most suitable one for deployment on the Jetson Nano. \
For this reason, each model is trained using its native augmentations, meaning the ones that were designed and optimized as part of its original architecture. \
It wouldn’t make sense to disable them or enforce a uniform setup across models, because what we want to observe is the maximum potential of each model, working in the way it was designed to perform best.

# Fine-Tuning Model

### YOLOv8n

We want to create the data.yaml file, which YOLO uses to know:
- the path to the training, validation, and test images
- the number of classes (nc)
- the names of the classes (names)

\
This file is used by YOLO to locate the images and their annotations.

In [5]:
# YAML dataset (edit routes)
data_yaml = """
train: /content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n/train/images
val:   /content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n/val/images
test:  /content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n/test/images

nc: 1
names: ['person']
"""
open('data.yaml', 'w').write(data_yaml)

260

Perfect, we have correctly written the data.yaml file for the AERALIS_YOLOv8n dataset. Let's continue with the loading of the model:

In [6]:
# Upload the pre-trained model we want to use as a starting point

model = YOLO('yolov8n.pt') # it is the model that will be fine-tuned on the custom dataset

100%|██████████| 6.25M/6.25M [00:00<00:00, 80.4MB/s]


We have now downloaded the pre-trained model from the official Ultralytics repository.

The **Batch size** is the number of images processed simultaneously in each training step. With 4-8GB of RAM, a batch size of 8 (or even less) is recommended, so we'll start with that value and reduce it if necessary.

**Early Stopping** is a technique that automatically stops the training process if the model stops improving after a certain number of epochs. This helps prevent overfitting and saves time.

**Workers** are the parallel processes used to load and preprocess data while the model is training. However, due to our limited resources, we’ll start with 2 workers, and if data loading errors occur, we'll reduce this number to 1 or even 0.


In [1]:
# To see the available GPU
import torch
print(torch.cuda.is_available()) # True = you have GPU --> if False then use device='cpu'
print(torch.cuda.device_count()) # Name of GPU

# If True and at least 1, you can use device=0.
# If you don't have GPU: use device='cpu' (much slower).
# Locally (not Colab): check with nvidia-smi from terminal.

True
1


Now that we have confirmation that the GPU is active we can train the model:

In [ ]:
# Fine‑tuning
results = model.train(
  data='data.yaml', # use the newly created yaml file
  epochs=100, # Maximum number of training epochs
  imgsz=640, # Image input size (recommended for YOLO).
  batch=8,  # Batch size
  patience=20, # Early stopping if the metrics do not improve for 20 epochs
  workers=2, # Number of workers for the dataloader
  device=0, # Use GPU 0 (or put 'cpu' if you don't have GPU)
  project='runs_finetune', # Folder where it will save the results of the experiments (the folder will be created automatically)
  name='person_yolov8n' # Subfolder/name specific to our experiment
)

# results -->  will contain metrics, logs, and the path of the best weights found during the training

We now want to evaluate the trained model using the Test set defined in data.yaml. \
YOLO does not compute standard accuracy, because in object detection True Negatives (TN) are not counted. Therefore, traditional accuracy is not applicable or useful.


So, we will compute:
- **Precision**: how correct your detected positives are
- **Recall**: how many of the real objects you detected
- **mAP50**: mean Average Precision with IoU ≥ 0.5 (how accurate the predictions are)
- **mAP50-95**: average over various IoU thresholds, a more strict metric
- **F1_score**: combination of precision and recall (you can compute it as: 2 * (P * R) / (P + R))

In [ ]:
# Evaluates the trained model using the TEST SET defined in data.yaml

metrics = model.val(data='data.yaml', split='test') # returns accuracy metrics (e.g., mAP, precision, recall, etc.) on the test set
print("Test metrics:", metrics)

Confidence (confidence) is the probability estimated by the model that a detected object is actually real (i.e., not a false positive).

In [ ]:
#  Inference (practical use of the fine-tuned model)

# I load the best weights found
best = results.best or f'runs_finetune/person_yolov8n/weights/best.pt' # or '...': is a manual backup in case results.best does not exist

model_inf = YOLO(best) # Creates a new model instance by loading the best weights

# Performs inference on one or more images, or on a video, by specifying the path in the source parameter.
# conf=0.25 → Confidence threshold for considering a detection valid.
preds = model_inf.predict(
  source='/content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n/test/images',
  conf=0.25
)
preds.show() # displays the predictions (eventually you can also save them)

### YOLOv11n

In [ ]:
# Carica il modello pre-addestrato che vogliamo usare come punto di partenza

model = YOLO('yolov11n.pt') # è il modello che verrà fine-tunato sul dataset custom

# Normalization and Data Augmentation

Per i modelli leggeri ottimizzati come quelli per Jetson Nano, la normalizzazione delle immagini è quasi sempre richiesta prima di passarle al modello.

I modelli in PyTorch lavorano SOLO con tensori, NON con immagini PIL o array NumPy.
- ToTensor() converte un’immagine (PIL o NumPy) in un tensore PyTorch di tipo float32, formato [C, H, W] (canale, altezza, larghezza).

- Inoltre, scala i valori dei pixel da [0,255] a [0,1] automaticamente.

La normalizzazione (Normalize) funziona SOLO su tensori.
La funzione Normalize(mean, std) richiede input già in formato tensore (float) e applica lo shift/scala canale per canale.

Se provi a normalizzare un’immagine PIL o un NumPy array direttamente, ottieni errore o comportamenti inattesi.

Quindi: la sequenza è SEMPRE
(Opzionale) Resize

ToTensor()   →  Converte e scala [0,255] in [0,1]

Normalize()  →  Normalizza ogni canale secondo mean/std richiesto dal modello


\
 In sintesi:
ToTensor è indispensabile, non è solo per PyTorch, ma anche perché la normalizzazione funziona SOLO su tensori, non su immagini raw!

La normalizzazione NON sostituisce ToTensor: lavora sopra i dati già convertiti.

La sequenza ToTensor() + normalizzazione è quasi sempre necessaria, ma i dettagli della normalizzazione (mean, std, range pixel) possono cambiare in base al modello.
Vediamo la situazione per i tuoi modelli:
-


# Fine-Tuning Models (Alexia)

Analizzo il codice di Alexia per avere uno spunto:

1. test_comptage_img.py
Scopo:
Carica un modello YOLO addestrato e conta quanti oggetti della classe 0 (qui chiamati "oiseaux" = uccelli, ma tu potresti adattare a "persone") vengono rilevati in una singola immagine.

In [ ]:
from ultralytics import YOLO # Importa la libreria Ultralytics YOLO

# Chargement du modèle entraîné
model = YOLO("../runs/detect/train4/weights/best.pt") # Carica il modello YOLO addestrato dal file best.pt (specificare il percorso giusto)

# Prédiction sur une image
results = model.predict("test4.jpeg") # Esegue la predizione sull'immagine "test4.jpeg"

# Compte des oiseaux (classe 0)
bird_count = sum(1 for cls in results[0].boxes.cls if int(cls) == 0) # Conta quante bounding box appartengono alla classe 0

print(f"Nombre d'oiseaux : {bird_count}") # Stampa il numero di oggetti (classe 0) rilevati


Nota:

Puoi cambiare "classe 0" con "persona" se il tuo modello rileva persone come classe 0.

2. test_comptage_video.py
Scopo:
Carica un modello YOLO addestrato, effettua il tracking (con ByteTrack) e conta quanti oggetti della classe 0 ("oiseaux") entrano in un rettangolo centrale all'interno di un video.
Annota la video con bounding box, ID, conta corrente e totale degli oggetti unici che sono entrati nel rettangolo.

In [ ]:
from ultralytics import YOLO                # Importa YOLO da Ultralytics
import cv2                                  # Importa OpenCV per gestione video e immagini
import os                                   # Importa os (qui non usato, ma spesso per path)

# Charger le modèle
model = YOLO("/Users/alexiagaido--amoros/Desktop/UPV-test/entrainement_serveur/runs/detect/train9/weights/best.pt")
# Carica il modello YOLO addestrato (specifica percorso)

# Chemin de la vidéo
video_path = "img_video/video_test_1.mp4"   # Path della video da analizzare
output_path = "output_video.mp4"            # Path della video annotata in output

# Distance des bords pour le rectangle de contact (en pixels)
border_distance = 50                        # Margine dai bordi (pixels) per il rettangolo centrale

# Ouvrir la vidéo
cap = cv2.VideoCapture(video_path)          # Apre la video
if not cap.isOpened():
    print("Erreur : Impossible d'ouvrir la vidéo")   # Se non apre la video, errore
    exit()

# Obtenir les propriétés de la vidéo
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))       # Ottiene larghezza frame
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))     # Ottiene altezza frame
fps = int(cap.get(cv2.CAP_PROP_FPS))                # Ottiene fps

# Définir les coordonnées du rectangle de contact
rect_x1 = border_distance                           # Coordinate x1 del rettangolo
rect_y1 = border_distance                           # Coordinate y1
rect_x2 = width - border_distance                   # Coordinate x2
rect_y2 = height - border_distance                  # Coordinate y2

# Configurer la sortie vidéo
fourcc = cv2.VideoWriter_fourcc(*"mp4v")            # Codec video per output
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))  # Oggetto per scrivere la video annotata

# Ensemble pour stocker les IDs uniques des oiseaux dans le rectangle
unique_bird_ids = set()                             # Insieme per salvare gli ID unici degli oggetti che sono passati nel rettangolo

while cap.isOpened():
    ret, frame = cap.read()                         # Leggi un frame
    if not ret:
        break

    # Effectuer l'inférence avec suivi
    results = model.track(frame, conf=0.5, tracker="bytetrack.yaml", persist=True)
    # Fa inferenza + tracking, usa ByteTrack, restituisce risultati con ID di tracking

    # Compter les oiseaux dans cette frame
    bird_count = 0
    if results[0].boxes.id is not None:             # Se ci sono ID di tracking
        for box, box_id in zip(results[0].boxes, results[0].boxes.id):   # Scorri bounding box e relativi ID
            # Vérifier si le centre de la bounding box est dans le rectangle
            x_center = (box.xyxy[0][0] + box.xyxy[0][2]) / 2            # Calcola centro x
            y_center = (box.xyxy[0][1] + box.xyxy[0][3]) / 2            # Calcola centro y
            if rect_x1 < x_center < rect_x2 and rect_y1 < y_center < rect_y2:   # Se centro box dentro rettangolo centrale
                unique_bird_ids.add(box_id.item())                      # Aggiungi ID a set (oggetti unici che sono passati)
                bird_count += 1                                         # Conta per questa frame

    # Annoter l'image avec les détections et IDs
    annotated_frame = results[0].plot()              # Disegna box e ID sul frame

    # Dessiner le rectangle de contact
    cv2.rectangle(
        annotated_frame,
        (rect_x1, rect_y1),
        (rect_x2, rect_y2),
        (255, 0, 0),  # Blu
        2,            # Spessore linea
    )

    # Afficher le nombre d'oiseaux dans cette frame et le total unique
    cv2.putText(
        annotated_frame,
        f"Oiseaux dans cette frame : {bird_count}",
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),  # Verde
        2,
    )
    cv2.putText(
        annotated_frame,
        f"Oiseaux uniques : {len(unique_bird_ids)}",
        (10, 60),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2,
    )

    # Écrire l'image annotée dans la vidéo de sortie
    out.write(annotated_frame)

    # Afficher l'image en temps réel
    cv2.imshow("YOLO Tracking", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):  # Premere 'q' per uscire
        break

# Afficher le total des oiseaux uniques détectés
print(f"Nombre total d'oiseaux uniques détectés dans la vidéo : {len(unique_bird_ids)}")

# Libérer les ressources
cap.release()
out.release()
cv2.destroyAllWindows()

Considerazioni tecniche
Classe 0: Il codice è pensato per oggetti "oiseaux" (uccelli) = classe 0. Se tu hai persone come classe 0, funziona identico.

Tracking (ByteTrack): Permette di assegnare un ID a ogni oggetto/persona che attraversa l’area, così da contarli solo una volta anche se si fermano/muovono nella scena.

Rettangolo di interesse: Conta solo gli oggetti il cui centro entra in una zona centrale, utile ad esempio per contare solo chi passa in una certa area (adattabile per ingressi, uscite, ecc).

Salvataggio video annotato: Il risultato è un video con box, ID e conteggi stampati sopra.